# Wordle solver
We need to make some assumptions to write an explicit solution, such as:

* Words are likely to be the solution proportional to their use in the corpus
  * In reality, there may be a non-linear relationship, e.g. square root
  * **My intuition is to use metric `letter_freq += 1 + math.sqrt(word_frequency)`** for each occurence of letter
* Based on the above, letters are likely to be in the solution proportional to their (weighted) use in the corpus
* Getting a "green" is more valuable than getting a "yellow" (take letter frequency at individual positions into account)
* etc.

A better solution may be to use reinforcement learning, so as to avoid making these assumptions.

In [1]:
import pandas as pd
import numpy as np
import string
from collections import defaultdict
import math

In [2]:
print('Preparing vocabulary...')

Preparing vocabulary...


In [3]:
# https://www.kaggle.com/datasets/rtatman/english-word-frequency?resource=download
words = pd.read_csv('./data/unigram_freq.csv')
az = string.ascii_lowercase

In [4]:
# Filter to 5 letter words
words = words[words['word'].str.len() == 5]
words['freq'] = words['count'] / words['count'].max()
print(len(words))
words.head()

39933


,word,count,freq
35,about,1226734006,1.000000
45,other,978481319,0.797631
56,which,810514085,0.660709
57,their,782849411,0.638157
62,there,701170205,0.571575


In [5]:
# Allowable wordle words
# https://github.com/tabatkins/wordle-list
allowed_words = pd.read_csv('./data/wordle-allowed-words.txt', header=None)
print(len(allowed_words))
allowed_words.head()

12947


,0
0,women
1,nikau
2,swack
3,feens
4,fyles


In [6]:
# Filter word frequency table to include allowed words only
words = words[words['word'].isin(set(allowed_words[0]))]
len(words)

8072

In [7]:
# Add the other allowed words but with count and freq 0
counted_words = set(words['word'])
for w in allowed_words[0]:
    if w not in counted_words:
        new_row = {'word': w, 'count':0, 'freq':0}
        words = words.append(new_row, ignore_index=True)
        
print(len(words))

12947


## 1. Find "best" words

In [8]:
# Letter frequency
letter_freq = defaultdict(int)
for _, w in words.iterrows():
    for char in w['word']:
#         Many ways to weight this:
#         letter_freq[char] += w['freq']
#         letter_freq[char] += 1
        letter_freq[char] += 1 + math.sqrt(w['freq'])

sorted_letters = sorted(letter_freq.items(), key=lambda x: -x[1])
sorted_letters[:5]

[('e', 6826.373093957733),
 ('s', 6797.120446066696),
 ('a', 6115.552312164332),
 ('o', 4536.092041527143),
 ('r', 4266.03717813045)]

In [9]:
# Find the best "starting words"
# Best: maximizing total letter_freq (but 0 score for repeated letter)
# This doesn't take into account letter position
word_goodness = defaultdict(float)
for w in allowed_words[0]:
    used_chars = set()
    for char in w:
        if char in used_chars:
            continue
        word_goodness[w] += letter_freq[char]
        used_chars.add(char)
        
sorted_words = sorted(word_goodness.items(), key=lambda x: -x[1])
sorted_words[:5]

[('arose', 28541.175071846355),
 ('soare', 28541.17507184635),
 ('aeros', 28541.17507184635),
 ('raise', 27847.75322556465),
 ('serai', 27847.75322556465)]

In [10]:
# Take letter position into account
# Letter frequency per position
letter_pos_freq = [defaultdict(float) for _ in range(5)]
for _, w in words.iterrows():
    for i, char in enumerate(w['word']):
#         Many ways to weight this:
#         letter_freq[char] += w['freq']
#         letter_freq[char] += 1
        letter_pos_freq[i][char] += 1 + math.sqrt(w['freq'])

sorted_pos_letters = [sorted(letter_pos_freq[i].items(), key=lambda x: -x[1]) for i in range(5)]
[s[0] for s in sorted_pos_letters]

[('s', 1599.4231123478824),
 ('a', 2308.278555858041),
 ('a', 1272.652282524225),
 ('e', 2382.654624241403),
 ('s', 4028.057199519846)]

In [11]:
# Find the best "starting words"
# Best: maximizing total letter_freq (but 0 score for repeated letter)
# This doesn't take into account letter position
word_goodness_pos = defaultdict(float)
for w in allowed_words[0]:
    used_chars = set()
    for i, char in enumerate(w):
#         if char in used_chars:
#             continue
        word_goodness_pos[w] += letter_pos_freq[i][char]
        used_chars.add(char)
        
sorted_words_pos = sorted(word_goodness_pos.items(), key=lambda x: -x[1])
sorted_words_pos[:5]

[('sores', 11378.663564416172),
 ('sanes', 11302.442739774659),
 ('sales', 11184.413237562803),
 ('sones', 11139.699481722342),
 ('soles', 11021.669979510487)]

In [12]:
word_good_df = pd.DataFrame(dict(word_goodness), index=['']).T
word_good_df.columns = ['overall']

In [13]:
word_good_df_pos = pd.DataFrame(dict(word_goodness_pos), index=['']).T
word_good_df_pos.columns = ['position']

In [14]:
word_good_df_all = word_good_df.join(word_good_df_pos)
word_good_df_all['avg'] = (
    (word_good_df_all['overall'] / word_good_df_all['overall'].max())
    + (word_good_df_all['position'] / word_good_df_all['position'].max())
) / 2
best_words = word_good_df_all.sort_values(['avg'], ascending=False)
best_words.head()

,overall,position,avg
tares,27396.857394,10781.425154,0.953709
lares,27460.499011,10532.511384,0.943887
cares,26082.927054,10887.691356,0.935361
pares,26061.732612,10820.055129,0.932017
dares,26511.353264,10636.135606,0.931812


## 2. Solver (hard mode)
This will always use existing clues. Note that this is not an optimal strategy for playing not in hard mode - where using a completely different set of letters is preferred for more information gain.

We can define a set of allowable letters for each position, which is initially all letters, then narrows.

Start by doing an example.

In [15]:
best_words

,overall,position,avg
tares,27396.857394,10781.425154,0.953709
lares,27460.499011,10532.511384,0.943887
cares,26082.927054,10887.691356,0.935361
pares,26061.732612,10820.055129,0.932017
dares,26511.353264,10636.135606,0.931812
...,...,...,...
infix,8299.023116,1674.794133,0.218980
kudzu,7023.637653,2179.129019,0.218799
oxbow,7550.825748,1440.363057,0.195572
immix,6154.652583,1853.214489,0.189254


In [16]:
green = defaultdict(int)
set(green.keys())

set()

In [32]:
best_word_list = best_words.index
best_word_array = np.array([np.array([c for c in w]) for w in best_word_list])

def use_clues(word, result):
    # Special case: Same letter is green and grey
    # Have to keep track of position of green chars
    green_idx = defaultdict(list)
    grey = set()
    for i, r in enumerate(result):
        char = word[i]
        if r == 'grey':
            grey.add(char)
        elif r == 'green':
            green_idx[char].append(i)
    green_and_grey = set(green_idx.keys()).intersection(grey)
    for i, r in enumerate(result):
        char = word[i]
        if r == 'green':
            a[i] = set([char])
        elif r == 'yellow':
            try:
                a[i].remove(char)
            except:
                continue
            b.add(char)
        elif r == 'grey':
            if char in green_and_grey:
                # Remove this option from all but the green positions
                for i, pos in enumerate(a):
                    if i in green_idx[char]:
                        continue
                    try:
                        pos.remove(char)
                    except:
                        continue
            else:
                # Remove from all positions
                for pos in a:
                    try:
                        pos.remove(char)
                    except:
                        continue
        

# Find all allowable words matching the constraints
def find_ok_words(a, b):
    a_met = (
        np.in1d(best_word_array[:,0], list(a[0]))
        & np.in1d(best_word_array[:,1], list(a[1]))
        & np.in1d(best_word_array[:,2], list(a[2]))
        & np.in1d(best_word_array[:,3], list(a[3]))
        & np.in1d(best_word_array[:,4], list(a[4]))
    )
#     print('meets a:', a_met.sum())
    if len(b) > 0:
        b_met_each = []
        for char in b:
            b_met_each.append(np.array([char in word for word in best_word_list]))
        b_met_all = np.array([True] * len(best_word_list))
        for b_met in b_met_each:
            b_met_all = b_met_all & b_met
    else:
        b_met_all = np.array([True] * len(best_word_list))
#     print('meets b:', b_met_all.sum())
    ok_words = best_word_array[a_met & b_met_all]
    return [''.join(w) for w in ok_words]

# Find the best clue word (most information)
def find_clue(a, b):
    ok_words = find_ok_words(a, b)
    # ok_words is already ranked by how useful the letters are
    return ok_words[0]

# Find the most likely solution word (most common)
def find_solution(a, b):
    ok_words = find_ok_words(a, b)
    print(len(ok_words), 'matching words.')
    # Get the most common word in ok_words
    ok_words_sorted = words[words['word'].isin(ok_words)]
    return ok_words_sorted['word'].values[0]

def find_clues_solutions(a, b):
    ok_words = find_ok_words(a, b)
    print(len(ok_words), 'matching words. Top picks:\n')
    # Get the most common word in ok_words
    most_common_words = words[words['word'].isin(ok_words)]
    most_common_words.index = most_common_words['word']
    most_common_words = most_common_words[['freq']]
    most_common_words.columns = ['Word Frequency']
    print('WORD FREQUENCY (most common words):')
    print(most_common_words.head())
    best_info_words = best_words.loc[ok_words][['avg']]
    best_info_words.columns = ['Information Gain']
    print('\nINFORMATION GAIN (frequent letters in common positions):')
    print(best_info_words.head())
    best_of_both = best_info_words.join(most_common_words)
    best_of_both['Combined Score'] = np.sqrt(best_of_both['Word Frequency']) * best_of_both['Information Gain']
    best_of_both = best_of_both.sort_values(['Combined Score'], ascending=False)
    print('\nCOMBINED SCORE: (sqrt(word_frequency) * info_gain)')
    print(best_of_both[['Combined Score', 'Information Gain', 'Word Frequency']].head(5))
    print()
    
# Shorthands to make entering results easier
sh = ['grey', 'yellow', 'green']

In [33]:
while True:
    # Allowed letters
    a = [set(az) for _ in range(5)]
    # Necessary letters
    b = set()
    
    quit = False
    
    print('WORDLE SOLVER')
    print('With no contraints, there are...')
    find_clues_solutions(a,b)

    for i in range(6):
        word = input(f'Guess {i+1} (or type "exit", "restart"): ')
        if word == 'restart':
            break
        elif word == 'exit':
            quit = True
            break
        result = input('Result? (0=grey, 1=yellow, 2=green:\n')
        result = [sh[int(s)] for s in result]
        use_clues(word, result)
#         print('Best solution word is:', find_solution(a, b))
#         print('Best clue word is:', find_clue(a,b))
        find_clues_solutions(a, b)
        print('')
        
    if quit:
        break

WORDLE SOLVER
With no contraints, there are...
12947 matching words. Top picks:

WORD FREQUENCY (most common words):
       Word Frequency
word                 
about        1.000000
other        0.797631
which        0.660709
their        0.638157
there        0.571575
INFORMATION GAIN (frequent letters in common positions):
       Information Gain
tares          0.953709
lares          0.943887
cares          0.935361
pares          0.932017
dares          0.931812
COMBINED SCORE: (sqrt(word_frequency) * info_gain)
       Combined Score  Information Gain  Word Frequency
other        0.471749          0.528214        0.797631
about        0.452289          0.452289        1.000000
games        0.422009          0.845055        0.249386
years        0.421771          0.803703        0.275399
their        0.418910          0.524393        0.638157
state        0.395993          0.651575        0.369358
pages        0.370894          0.849213        0.190751
first        0.370403        

In [40]:
best_words

,overall,position,avg
tares,27396.857394,10781.425154,0.953709
lares,27460.499011,10532.511384,0.943887
cares,26082.927054,10887.691356,0.935361
pares,26061.732612,10820.055129,0.932017
dares,26511.353264,10636.135606,0.931812
...,...,...,...
infix,8299.023116,1674.794133,0.218980
kudzu,7023.637653,2179.129019,0.218799
oxbow,7550.825748,1440.363057,0.195572
immix,6154.652583,1853.214489,0.189254


In [48]:
best_of_both = best_words.copy()[['overall', 'position']]
best_of_both.columns = ['Information Gain', 'Word Frequency']
best_of_both['word'] = best_of_both.index
best_of_both = best_of_both.sort_values(['Information Gain']).reset_index().drop(['index'],axis=1)
best_of_both['info_idx'] = best_of_both.index
best_of_both = best_of_both.sort_values(['Word Frequency']).reset_index().drop(['index'],axis=1)
best_of_both['freq_idx'] = best_of_both.index

info_weight = 0.5
freq_weight = 0.5
best_of_both['combined_flat'] = best_of_both['info_idx']*info_weight + best_of_both['freq_idx']*freq_weight
best_of_both = best_of_both.sort_values(['combined_flat'], ascending=False).reset_index().drop(['index'],axis=1)
print(best_of_both.head())

   Information Gain  Word Frequency   word  info_idx  freq_idx  combined_flat
0      27396.857394    10781.425154  tares     12917     12935        12926.0
1      27460.499011    10532.511384  lares     12934     12913        12923.5
2      26511.353264    10636.135606  dares     12882     12924        12903.0
3      27460.499011    10226.761744  rales     12931     12875        12903.0
4      26586.236196    10424.431569  tales     12899     12899        12899.0
